In [1]:
import pandas as pd
train=pd.read_csv("train.tsv",sep="\t")
test=pd.read_csv("test.tsv",sep="\t")

In [2]:
from bs4 import BeautifulSoup

In [3]:
# The Natural Language Toolkit, or more commonly NLTK, is a suite of libraries and programs for symbolic and 
# statistical natural language processing for English written in the Python programming language.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from bs4 import BeautifulSoup
import re

#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm

In [4]:
from keras.utils import to_categorical
import random
from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential

#set random seed for the session and also for tensorflow that runs in background for keras
set_random_seed(123)
random.seed(123)

Using TensorFlow backend.


In [5]:
train.shape


(156060, 4)

In [6]:
test.shape

(66292, 3)

In [7]:
def clean_sentences(df):
    reviews = []

    for sent in tqdm(df['Phrase']):
        
        #remove html content
        review_text = BeautifulSoup(sent).get_text()
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        #tokenize the sentences
        words = word_tokenize(review_text.lower())
    
        #lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)

In [8]:
#cleaned reviews for both train and test set retrieved
train_sentences = clean_sentences(train)
test_sentences = clean_sentences(test)
print(len(train_sentences))
print(len(test_sentences))

  0%|                                                                            | 1/156060 [00:01<56:40:48,  1.31s/it]D:\Anaconda\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
  0%|▎                                                                         | 628/156060 [00:01<27:40:07,  1.56it/s]D:\Anaconda\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
100%|██████████████████████████████████████████████████████████████████████████| 66292/66292 [00:23<00:00, 2877.32it/s]


156060
66292


In [9]:
target=train.Sentiment.values
y_target=to_categorical(target)
num_classes=y_target.shape[1]

In [10]:
X_train,X_val,y_train,y_val=train_test_split(train_sentences,y_target,test_size=0.2,stratify=y_target)


In [11]:
#It is needed for initializing tokenizer of keras and subsequent padding

unique_words = set()
len_max = 0

for sent in tqdm(X_train):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)
        
#length of the list of unique_words gives the no of unique words
print(len(list(unique_words)))
print(len_max)


100%|██████████████████████████████████████████████████████████████████████| 124848/124848 [00:00<00:00, 767984.84it/s]


13737
48


In [12]:
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))

#texts_to_sequences(texts)

    # Arguments- texts: list of texts to turn to sequences.
    #Return: list of sequences (one per text input).
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(test_sentences)

#padding done to equalize the lengths of all input reviews. LSTM networks needs all inputs to be same length.
#Therefore reviews lesser than max length will be made equal using extra zeros at end. This is padding.

X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
X_test = sequence.pad_sequences(X_test, maxlen=len_max)

print(X_train.shape,X_val.shape,X_test.shape)


(124848, 48) (31212, 48) (66292, 48)


In [13]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 2)
callback = [early_stopping]

In [14]:
#Model using Keras LSTM

#Multilayer Perceptron (MLP) for multi-class softmax classification:
#Let’s build what’s probably the most popular type of model in NLP at the moment: Long Short Term Memory network. 
#This architecture is specially designed to work on sequence data.
#It fits perfectly for many NLP tasks like tagging and text classification.
#It treats the text as a sequence rather than a bag of words or as ngrams.

#Here’s a possible model definition:

model=Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()


W0207 21:15:39.656015  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0207 21:15:39.660007  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0207 21:15:39.663025  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0207 21:15:40.060930  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0207 21:15:40.068908  3156 deprecation.py:506] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 300)           4121100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 48, 128)           219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 505       
Total params: 4,397,161
Trainable params: 4,397,161
Non-trainable params: 0
_________________________________________________________________


In [15]:
#This is done for learning purpose only. One can play around with different hyper parameters combinations
#and try increase the accuracy even more. For example, a different learning rate, an extra dense layer 
# before output layer, etc. Cross validation could be used to evaluate the model and grid search 
# further to find unique combination of parameters that give maximum accuracy. This model has a validation
#accuracy of around 64.6%
history=model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=1, batch_size=256, verbose=1, callbacks=callback)

W0207 21:17:57.705650  3156 deprecation.py:323] From D:\Anaconda\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0207 21:17:59.030078  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0207 21:17:59.170703  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0207 21:17:59.288420  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Train on 124848 samples, validate on 31212 samples
Epoch 1/1


W0207 21:17:59.295369  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0207 21:17:59.297367  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0207 21:17:59.478879  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0207 21:17:59.479877  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

W0207 21:17:59.636457  3156 module_wrapper.py:139] From D:\Anaconda\lib\site-packages\keras\backend\te

 52224/124848 [===========>..................] - ETA: 16:12 - loss: 1.6095 - acc: 0.22 - ETA: 9:49 - loss: 1.5632 - acc: 0.3516 - ETA: 7:43 - loss: 1.5033 - acc: 0.399 - ETA: 6:40 - loss: 1.4551 - acc: 0.426 - ETA: 6:06 - loss: 1.4175 - acc: 0.442 - ETA: 5:37 - loss: 1.3853 - acc: 0.449 - ETA: 5:14 - loss: 1.3735 - acc: 0.457 - ETA: 4:58 - loss: 1.3788 - acc: 0.450 - ETA: 4:45 - loss: 1.3724 - acc: 0.451 - ETA: 4:34 - loss: 1.3589 - acc: 0.457 - ETA: 4:26 - loss: 1.3477 - acc: 0.463 - ETA: 4:19 - loss: 1.3412 - acc: 0.469 - ETA: 4:13 - loss: 1.3328 - acc: 0.475 - ETA: 4:08 - loss: 1.3258 - acc: 0.479 - ETA: 4:03 - loss: 1.3222 - acc: 0.483 - ETA: 3:59 - loss: 1.3262 - acc: 0.483 - ETA: 3:56 - loss: 1.3211 - acc: 0.486 - ETA: 3:53 - loss: 1.3203 - acc: 0.488 - ETA: 3:50 - loss: 1.3222 - acc: 0.486 - ETA: 3:47 - loss: 1.3188 - acc: 0.487 - ETA: 3:44 - loss: 1.3180 - acc: 0.487 - ETA: 3:42 - loss: 1.3151 - acc: 0.486 - ETA: 3:40 - loss: 1.3087 - acc: 0.488 - ETA: 3:39 - loss: 1.3056 - acc

105216/124848 [========================>.....] - ETA: 2:03 - loss: 1.1161 - acc: 0.559 - ETA: 2:03 - loss: 1.1152 - acc: 0.559 - ETA: 2:02 - loss: 1.1145 - acc: 0.559 - ETA: 2:02 - loss: 1.1145 - acc: 0.559 - ETA: 2:02 - loss: 1.1143 - acc: 0.559 - ETA: 2:01 - loss: 1.1137 - acc: 0.560 - ETA: 2:01 - loss: 1.1132 - acc: 0.560 - ETA: 2:00 - loss: 1.1126 - acc: 0.560 - ETA: 2:00 - loss: 1.1118 - acc: 0.560 - ETA: 1:59 - loss: 1.1113 - acc: 0.560 - ETA: 1:59 - loss: 1.1104 - acc: 0.561 - ETA: 1:59 - loss: 1.1092 - acc: 0.561 - ETA: 1:58 - loss: 1.1090 - acc: 0.561 - ETA: 1:58 - loss: 1.1085 - acc: 0.561 - ETA: 1:57 - loss: 1.1079 - acc: 0.562 - ETA: 1:57 - loss: 1.1069 - acc: 0.562 - ETA: 1:56 - loss: 1.1061 - acc: 0.562 - ETA: 1:56 - loss: 1.1056 - acc: 0.562 - ETA: 1:56 - loss: 1.1047 - acc: 0.563 - ETA: 1:55 - loss: 1.1040 - acc: 0.563 - ETA: 1:55 - loss: 1.1033 - acc: 0.563 - ETA: 1:54 - loss: 1.1023 - acc: 0.563 - ETA: 1:54 - loss: 1.1017 - acc: 0.563 - ETA: 1:53 - loss: 1.1013 - acc:

124848/124848 [==============================] - ETA: 33s - loss: 1.0196 - acc: 0.59 - ETA: 33s - loss: 1.0193 - acc: 0.59 - ETA: 33s - loss: 1.0191 - acc: 0.59 - ETA: 32s - loss: 1.0190 - acc: 0.59 - ETA: 32s - loss: 1.0188 - acc: 0.59 - ETA: 31s - loss: 1.0185 - acc: 0.59 - ETA: 31s - loss: 1.0182 - acc: 0.59 - ETA: 30s - loss: 1.0178 - acc: 0.59 - ETA: 30s - loss: 1.0176 - acc: 0.59 - ETA: 29s - loss: 1.0173 - acc: 0.59 - ETA: 29s - loss: 1.0170 - acc: 0.59 - ETA: 29s - loss: 1.0167 - acc: 0.59 - ETA: 28s - loss: 1.0163 - acc: 0.59 - ETA: 28s - loss: 1.0160 - acc: 0.59 - ETA: 27s - loss: 1.0157 - acc: 0.59 - ETA: 27s - loss: 1.0153 - acc: 0.59 - ETA: 26s - loss: 1.0152 - acc: 0.59 - ETA: 26s - loss: 1.0149 - acc: 0.59 - ETA: 25s - loss: 1.0145 - acc: 0.59 - ETA: 25s - loss: 1.0142 - acc: 0.59 - ETA: 24s - loss: 1.0141 - acc: 0.59 - ETA: 24s - loss: 1.0135 - acc: 0.59 - ETA: 24s - loss: 1.0132 - acc: 0.59 - ETA: 23s - loss: 1.0129 - acc: 0.59 - ETA: 23s - loss: 1.0127 - acc: 0.59 - E